In [1]:
from koselleck import *

In [2]:
df=pd.read_csv('data/data.vector_scores_across_models.csv')
df

,corpus,period,run,word,Abs-Conc.Median,Woman-Man.S,Woman-Man.M,Good-Bad.S,Future-Past.M,Future-Past.S,Pos-Neg.HGI
0,bpo,1680-1690,run_01,sir,2.276922,1.187223,0.571455,-1.347516,-2.357921,-2.357921,-2.602805
1,bpo,1680-1690,run_01,hat,0.191133,-1.315730,-1.125410,1.027656,-2.987615,-2.987615,-2.412253
2,bpo,1680-1690,run_01,ore,1.318373,0.125374,0.136961,-0.949728,-1.270908,-1.270908,-2.382637
3,bpo,1680-1690,run_01,tic,1.276977,-0.074494,-0.176665,-0.381085,-1.342881,-1.342881,-2.293757
4,bpo,1680-1690,run_01,tit,1.449162,0.241363,0.070920,-0.925371,-0.888443,-0.888443,-2.231821
...,...,...,...,...,...,...,...,...,...,...,...
2301715,bpo,1890-1900,run_01,grace,-0.949842,1.257025,1.841225,3.750016,1.634311,0.895173,3.630201
2301716,bpo,1890-1900,run_01,excellence,-1.178021,-0.423361,-0.711680,2.708450,1.722180,1.238616,3.791792
2301717,bpo,1890-1900,run_01,ability,-1.886885,-1.638629,-1.448200,3.212223,2.343172,2.426203,3.799819
2301718,bpo,1890-1900,run_01,gift,-0.921761,0.382866,0.587103,3.130570,1.124278,1.574570,4.002125


In [3]:
for i,gdf in df.groupby('word'): pass

In [51]:
def compare_means(df,num_proc=1):
    odf=pmap_groups(
        do_compare_means,
        df.groupby('word'),
        num_proc=num_proc
    ).reset_index()
    return odf.set_index(['vector','word']).sort_values('mean_diff_t_abs',ascending=False)
    
def do_compare_means(gdf,before=1770, after=1830, ymin=1700, ymax=1900, truns=25, trunsize=100):
    gdf['year']=[int(str(period)[:4]) for period in gdf.period]
    gdf=gdf.query(f'{ymin}<=year<{ymax}')
    
    gdf_before = gdf.query(f'year<{before}')
    gdf_after = gdf.query(f'year>={after}')
    if not len(gdf_before) or not len(gdf_after): return pd.DataFrame()
    
    qcols = [c for c in gdf.select_dtypes('number').columns if c[0]==c[0].upper()]
    
    o=[]
    for qcol in qcols:
        for trun in range(truns):
            try:
                vals1=gdf_before[qcol].dropna().sample(n=trunsize,replace=True)
                vals2=gdf_after[qcol].dropna().sample(n=trunsize,replace=True)
            except ValueError as e:
                print(gdf.year.describe())
                display(gdf_before)
                display(gdf_after)
                
                print('!!',e)
                stop
                continue
            t,p=ttest_ind(vals1.values, vals2.values)
            o+=[{
                'vector':qcol,
                'mean1':vals1.mean(),
                'mean2':vals2.mean(),
                'mean_diff':vals2.mean() - vals1.mean(),
                'mean_diff_t':t,
                'mean_diff_p':p,
                'mean_diff_t_abs':abs(t),
                'len1':len(vals1),
                'len2':len(vals2),
                'trun':trun,
                'year1':gdf_before.year.median(),
                'year2':gdf_after.year.median(),
            }]
    if not o: return pd.DataFrame()
    odf=pd.DataFrame(o).groupby('vector').mean().reset_index()
    return odf.sort_values('mean_diff_t_abs',ascending=False).dropna()

In [52]:
round(do_compare_means(gdf),4)

,vector,mean1,mean2,mean_diff,mean_diff_t,mean_diff_p,mean_diff_t_abs,len1,len2,trun,year1,year2
0,Abs-Conc.Median,-0.6667,-0.0643,0.6023,-18.9188,0.0000,18.9188,100,100,12,1760.0,1850.0
1,Future-Past.M,0.6820,0.0908,-0.5911,15.4324,0.0000,15.4324,100,100,12,1760.0,1850.0
2,Future-Past.S,1.2543,0.9124,-0.3419,9.2745,0.0000,9.2745,100,100,12,1760.0,1850.0
5,Woman-Man.M,-0.9331,-0.5469,0.3861,-7.9390,0.0000,7.9390,100,100,12,1760.0,1850.0
4,Pos-Neg.HGI,1.5296,1.6944,0.1649,-4.5629,0.0029,4.5629,100,100,12,1760.0,1850.0
6,Woman-Man.S,-1.0405,-1.1406,-0.1001,1.9958,0.1625,1.9958,100,100,12,1760.0,1850.0
3,Good-Bad.S,0.7179,0.6645,-0.0534,1.0049,0.3840,1.0539,100,100,12,1760.0,1850.0


In [53]:
odf = compare_means(df, num_proc=4)

Mapping do_compare_means [x4]: 100%|██████████| 6001/6001 [04:42<00:00, 21.24it/s]


In [54]:
odf.to_csv('data/data.vector_scores_across_models.diff_means.csv')

In [59]:
odf

mean1     mean2  mean_diff  mean_diff_t  \
vector          word                                                      
Future-Past.M   masterpiece -2.894139  1.698011   4.592150  -125.012012   
Woman-Man.M     canning      2.160463 -2.452595  -4.613058   114.957748   
                marguerite  -0.308491  3.913045   4.221536  -100.232333   
Woman-Man.S     sportsman    0.563263 -3.041994  -3.605258    94.249744   
Abs-Conc.Median reaction     0.382831 -1.790138  -2.172969    94.060078   
...                               ...       ...        ...          ...   
Woman-Man.S     flu         -0.080512 -0.094568  -0.014055     0.228674   
Good-Bad.S      charity      3.359852  3.358819  -0.001033     0.029066   
Pos-Neg.HGI     situation    1.075530  1.073578  -0.001952     0.028954   
Good-Bad.S      regent      -0.042520 -0.012956   0.029565    -0.341687   
Woman-Man.S     border       0.970854  0.965609  -0.005245     0.094419   

                               mean_diff_p  mean_diff_t_abs   len1   len2  \
vector          word                                                        
Future-Past.M   masterpiece  9.459297e-181       125.012012  100.0  100.0   
Woman-Man.M     canning      4.030263e-172       114.957748  100.0  100.0   
                marguerite   2.600817e-163       100.232333  100.0  100.0   
Woman-Man.S     sportsman    2.179887e-161        94.249744  100.0  100.0   
Abs-Conc.Median reaction     8.206376e-161        94.060078  100.0  100.0   
...                                    ...              ...    ...    ...   
Woman-Man.S     flu           6.457189e-01         0.513926  100.0  100.0   
Good-Bad.S      charity       6.468336e-01         0.501638  100.0  100.0   
Pos-Neg.HGI     situation     6.566167e-01         0.495728  100.0  100.0   
Good-Bad.S      regent        6.513877e-01         0.488567  100.0  100.0   
Woman-Man.S     border        6.578176e-01         0.462532  100.0  100.0   

                             trun   year1   year2  
vector          word                               
Future-Past.M   masterpiece  12.0  1730.0  1850.0  
Woman-Man.M     canning      12.0  1755.0  1850.0  
                marguerite   12.0  1740.0  1850.0  
Woman-Man.S     sportsman    12.0  1755.0  1850.0  
Abs-Conc.Median reaction     12.0  1750.0  1850.0  
...                           ...     ...     ...  
Woman-Man.S     flu          12.0  1730.0  1850.0  
Good-Bad.S      charity      12.0  1730.0  1850.0  
Pos-Neg.HGI     situation    12.0  1730.0  1850.0  
Good-Bad.S      regent       12.0  1730.0  1850.0  
Woman-Man.S     border       12.0  1735.0  1850.0  

[36946 rows x 11 columns]

In [60]:
odf.head(25)

mean1     mean2  mean_diff  mean_diff_t  \
vector          word                                                         
Future-Past.M   masterpiece    -2.894139  1.698011   4.592150  -125.012012   
Woman-Man.M     canning         2.160463 -2.452595  -4.613058   114.957748   
                marguerite     -0.308491  3.913045   4.221536  -100.232333   
Woman-Man.S     sportsman       0.563263 -3.041994  -3.605258    94.249744   
Abs-Conc.Median reaction        0.382831 -1.790138  -2.172969    94.060078   
Pos-Neg.HGI     tact           -0.654908  2.714051   3.368960   -88.367845   
Woman-Man.M     bouquet        -1.130965  3.419744   4.550709   -86.529624   
Abs-Conc.Median tact            0.568838 -1.466417  -2.035255    85.696342   
Woman-Man.S     professorship   0.650575 -2.925550  -3.576125    85.571536   
Abs-Conc.Median transportation  0.717491 -1.208089  -1.925580    85.481129   
                bracelet       -0.131146  1.607182   1.738328   -84.872273   
                anomaly         0.077316 -1.796452  -1.873769    84.155383   
                admixture       0.674435 -0.663732  -1.338167    83.851897   
                fostering      -0.174822 -1.632853  -1.458030    81.561037   
                precision      -1.770938 -0.836575   0.934363   -81.464325   
Pos-Neg.HGI     professorship  -0.148226  2.246156   2.394382   -78.054398   
                depravity       1.803456 -0.962736  -2.766193    77.426325   
                duplicity       1.426360 -1.145914  -2.572274    77.370772   
Woman-Man.S     astronomer      0.158032 -2.323358  -2.481390    75.932416   
Pos-Neg.HGI     fiend           0.263156 -2.457884  -2.721040    75.269159   
Abs-Conc.Median irritation     -0.112022 -1.419556  -1.307534    75.118770   
                credence       -0.367708 -2.031437  -1.663729    74.153823   
Pos-Neg.HGI     durability      0.419932  2.226717   1.806785   -73.977161   
Abs-Conc.Median apathy         -0.493783 -2.405504  -1.911721    73.346624   
Future-Past.M   salon          -2.661505  1.303062   3.964567   -72.009028   

                                  mean_diff_p  mean_diff_t_abs   len1   len2  \
vector          word                                                           
Future-Past.M   masterpiece     9.459297e-181       125.012012  100.0  100.0   
Woman-Man.M     canning         4.030263e-172       114.957748  100.0  100.0   
                marguerite      2.600817e-163       100.232333  100.0  100.0   
Woman-Man.S     sportsman       2.179887e-161        94.249744  100.0  100.0   
Abs-Conc.Median reaction        8.206376e-161        94.060078  100.0  100.0   
Pos-Neg.HGI     tact            2.428655e-153        88.367845  100.0  100.0   
Woman-Man.M     bouquet         3.918364e-154        86.529624  100.0  100.0   
Abs-Conc.Median tact            1.853657e-147        85.696342  100.0  100.0   
Woman-Man.S     professorship   8.317047e-153        85.571536  100.0  100.0   
Abs-Conc.Median transportation  7.910696e-153        85.481129  100.0  100.0   
                bracelet        1.776897e-152        84.872273  100.0  100.0   
                anomaly         2.365286e-153        84.155383  100.0  100.0   
                admixture       1.983035e-151        83.851897  100.0  100.0   
                fostering       9.330364e-151        81.561037  100.0  100.0   
                precision       6.417062e-150        81.464325  100.0  100.0   
Pos-Neg.HGI     professorship   3.804724e-138        78.054398  100.0  100.0   
                depravity       3.779753e-145        77.426325  100.0  100.0   
                duplicity       6.814133e-145        77.370772  100.0  100.0   
Woman-Man.S     astronomer      8.367177e-142        75.932416  100.0  100.0   
Pos-Neg.HGI     fiend           1.757633e-139        75.269159  100.0  100.0   
Abs-Conc.Median irritation      5.393842e-140        75.118770  100.0  100.0   
                credence        4.337629e-140        74.153823  100.0  100.0   
Pos-Neg.

In [61]:
odf.groupby('vector').head(5).sort_values(['vector','mean_diff_t_abs'], ascending=[1,0])

mean1     mean2  mean_diff  mean_diff_t  \
vector          word                                                         
Abs-Conc.Median reaction        0.382831 -1.790138  -2.172969    94.060078   
                tact            0.568838 -1.466417  -2.035255    85.696342   
                transportation  0.717491 -1.208089  -1.925580    85.481129   
                bracelet       -0.131146  1.607182   1.738328   -84.872273   
                anomaly         0.077316 -1.796452  -1.873769    84.155383   
Future-Past.M   masterpiece    -2.894139  1.698011   4.592150  -125.012012   
                salon          -2.661505  1.303062   3.964567   -72.009028   
                artist         -0.792870  2.647922   3.440792   -66.184201   
                concerto       -1.125225  1.630904   2.756128   -63.099501   
                quickness      -0.541526  2.122462   2.663988   -62.892470   
Future-Past.S   adhesion        0.495499  3.193176   2.697677   -62.096382   
                approximation   0.250595  2.043782   1.793187   -60.584879   
                nephew         -0.847873  1.981378   2.829251   -60.157048   
                bequest         0.112002  2.523009   2.411007   -55.102394   
                hum             0.018025 -1.953957  -1.971981    54.647108   
Good-Bad.S      depravity       1.807197 -1.051281  -2.858478    62.616010   
                thoroughfare    1.295944 -1.343876  -2.639820    61.378464   
                insanity       -0.021719 -2.748618  -2.726899    61.230702   
                dissipation     1.060038 -1.125309  -2.185347    52.768790   
                domination      0.759460 -1.406489  -2.165949    52.548317   
Pos-Neg.HGI     tact           -0.654908  2.714051   3.368960   -88.367845   
                professorship  -0.148226  2.246156   2.394382   -78.054398   
                depravity       1.803456 -0.962736  -2.766193    77.426325   
                duplicity       1.426360 -1.145914  -2.572274    77.370772   
                fiend           0.263156 -2.457884  -2.721040    75.269159   
Woman-Man.M     canning         2.160463 -2.452595  -4.613058   114.957748   
                marguerite     -0.308491  3.913045   4.221536  -100.232333   
                bouquet        -1.130965  3.419744   4.550709   -86.529624   
                adhesion        0.893006 -1.510941  -2.403947    61.259243   
                partnership     1.677347 -1.353708  -3.031056    54.084456   
Woman-Man.S     sportsman       0.563263 -3.041994  -3.605258    94.249744   
                professorship   0.650575 -2.925550  -3.576125    85.571536   
                astronomer      0.158032 -2.323358  -2.481390    75.932416   
                marguerite     -0.067973  3.950066   4.018040   -71.822540   
                laboratory      1.802715 -1.527559  -3.330274    67.136209   

                                  mean_diff_p  mean_diff_t_abs   len1   len2  \
vector          word                                                           
Abs-Conc.Median reaction        8.206376e-161        94.060078  100.0  100.0   
                tact            1.853657e-147        85.696342  100.0  100.0   
                transportation  7.910696e-153        85.481129  100.0  100.0   
                bracelet        1.776897e-152        84.872273  100.0  100.0   
                anomaly         2.365286e-153        84.155383  100.0  100.0   
Future-Past.M   masterpiece     9.459297e-181       125.012012  100.0  100.0   
                salon           5.578085e-137        72.009028  100.0  100.0   
                artist          4.370605e-131        66.184201  100.0  100.0   
                concerto        3.387087e-122        63.099501  100.0  100.0   
                quickness       2.044792e-125        62.892470  100.0  100.0   
Future-Past.S   adhesion        3.770032e-123        62.096382  100.0  100.0   
                approximation   6.560792e-118        60.584879  100.0  100.0   
                nephew      

In [58]:
odf.loc['Abs-Conc.Median'].head(25)

,mean1,mean2,mean_diff,mean_diff_t,mean_diff_p,mean_diff_t_abs,len1,len2,trun,year1,year2
word,,,,,,,,,,,
reaction,0.382831,-1.790138,-2.172969,94.060078,8.206376e-161,94.060078,100.0,100.0,12.0,1750.0,1850.0
tact,0.568838,-1.466417,-2.035255,85.696342,1.853657e-147,85.696342,100.0,100.0,12.0,1745.0,1850.0
transportation,0.717491,-1.208089,-1.925580,85.481129,7.910696e-153,85.481129,100.0,100.0,12.0,1735.0,1850.0
bracelet,-0.131146,1.607182,1.738328,-84.872273,1.776897e-152,84.872273,100.0,100.0,12.0,1750.0,1850.0
anomaly,0.077316,-1.796452,-1.873769,84.155383,2.365286e-153,84.155383,100.0,100.0,12.0,1740.0,1850.0
admixture,0.674435,-0.663732,-1.338167,83.851897,1.983035e-151,83.851897,100.0,100.0,12.0,1750.0,1850.0
fostering,-0.174822,-1.632853,-1.458030,81.561037,9.330364e-151,81.561037,100.0,100.0,12.0,1760.0,1850.0
precision,-1.770938,-0.836575,0.934363,-81.464325,6.417062e-150,81.464325,100.0,100.0,12.0,1755.0,1850.0
irritation,-0.112022,-1.419556,-1.307534,75.118770,5.393842e-140,75.118770,100.0,100.0,12.0,1755.0,1850.0


## New dfpiv

In [ ]:
df=pd.read_csv('data/')